In [12]:
import requests
import time

# commander_input = "sheoldred the apocalypse"
commander_input = "Rocco, Cabaretti Caterer"
# num_ramp = 10
# num_draw = 10
# num_removal = 10
num_ramp = 15
num_draw = 5
num_removal = 10

HEADERS = {
    "User-Agent": "deck-builder-bot/1.0",
    "Accept": "application/json"
}

def get_commander(name):
    url = "https://api.scryfall.com/cards/named"
    params = {
        "exact": name
    }

    r = requests.get(url, params=params, headers=HEADERS)
    r.raise_for_status()
    return r.json()


def fetch_all_cards(query):
    url = "https://api.scryfall.com/cards/search"
    params = {"q": query}
    print(query)
    cards = []
    card_index = 0
    while True:
        
        r = requests.get(url, params=params, headers=HEADERS)
        r.raise_for_status()
        data = r.json()
        card_index += len(data["data"]) 
        total_cards = data["total_cards"]
        percentage = float(card_index)/data["total_cards"] * 100
        print(f"current index {card_index}, total cards {total_cards}, percentage {percentage}")

        cards.extend(data["data"])

        if not data.get("has_more"):
            break

        # IMPORTANT: next_page already includes query params
        url = data["next_page"]
        params = None
        time.sleep(0.2) # dont want to get rate limited
    return cards


In [13]:
commander = get_commander(commander_input)

In [14]:
color_identity = commander["color_identity"]
print(commander['name'], color_identity)  # e.g. ['G', 'U', 'B', 'W']
def color_identity_query(ci):
    if not ci:
        return "ci=0"   # colorless commanders (Kozilek, etc.)
    return f"ci<={''.join(ci)}"
cid_string = color_identity_query(color_identity)
print(cid_string)

Rocco, Cabaretti Caterer ['G', 'R', 'W']
ci<=GRW


In [15]:
QUERIES = {
    "ramp" : f"usd<5 function:ramp {cid_string} -is:playtest -name:\"Sol Ring\" cmc<5 -type:land -set:UNF f:commander -is:gamechanger -o:snow",
    "draw" : f"usd<5 function:draw {cid_string} -is:playtest cmc<5 -type:land -set:UNF f:commander -is:gamechanger -o:snow",
    "removal" : f"usd<5 function:removal {cid_string} -is:playtest -type:land -set:UNF f:commander -is:gamechanger -o:snow"
}

ramp_cards    = fetch_all_cards(QUERIES["ramp"])
draw_cards    = fetch_all_cards(QUERIES["draw"])
removal_cards = fetch_all_cards(QUERIES["removal"])

usd<5 function:ramp ci<=GRW -name:"Sol Ring" cmc<5 -type:land -set:UNF f:commander -is:gamechanger -o:snow
current index 175, total cards 950, percentage 18.421052631578945
current index 350, total cards 950, percentage 36.84210526315789
current index 525, total cards 950, percentage 55.26315789473685
current index 700, total cards 950, percentage 73.68421052631578
current index 875, total cards 950, percentage 92.10526315789474
current index 950, total cards 950, percentage 100.0
usd<5 function:draw ci<=GRW cmc<5 -type:land -set:UNF f:commander -is:gamechanger -o:snow
current index 175, total cards 1032, percentage 16.95736434108527
current index 350, total cards 1032, percentage 33.91472868217054
current index 525, total cards 1032, percentage 50.872093023255815
current index 700, total cards 1032, percentage 67.82945736434108
current index 875, total cards 1032, percentage 84.78682170542635
current index 1032, total cards 1032, percentage 100.0
usd<5 function:removal ci<=GRW -type:l

In [16]:
import random

def pick_random(cards, n):
    if len(cards) < n:
        raise ValueError(f"Not enough cards to sample {n}")
    return random.sample(cards, n)



chosen_ramp    = pick_random(ramp_cards, num_ramp)
chosen_draw    = pick_random(draw_cards, num_draw)
chosen_removal = pick_random(removal_cards, num_removal)

In [17]:
COLOR_TO_BASIC = {
    "W": "Plains",
    "U": "Island",
    "B": "Swamp",
    "R": "Mountain",
    "G": "Forest"
}

def get_basic_lands(color_identity, total_basics=16):
    if not color_identity:
        return ["Wastes"] * total_basics

    basics = []
    per_color = total_basics // len(color_identity)
    remainder = total_basics % len(color_identity)

    for c in color_identity:
        basics.extend([COLOR_TO_BASIC[c]] * per_color)

    for i in range(remainder):
        basics.append(COLOR_TO_BASIC[color_identity[i]])

    return basics

def multicolor_land_query(ci):
    return (
        f"type:land -type:basic legal:commander -is:gamechanger -is:playtest "
        f"ci<={''.join(ci)} "
        f"o:/add .*[{''.join(ci)}].*[{''.join(ci)}]/"
    )


FETCH_QUERY = (
    "type:land legal:commander "
    "(o:search or o:sacrifice) -is:gamechanger -is:playtest "
    "-o:add"
)

multicolor_lands = fetch_all_cards(multicolor_land_query(color_identity))
fetch_lands = fetch_all_cards(FETCH_QUERY)

type:land -type:basic legal:commander -is:gamechanger ci<=GRW o:/add .*[GRW].*[GRW]/
current index 175, total cards 202, percentage 86.63366336633663
current index 202, total cards 202, percentage 100.0
type:land legal:commander (o:search or o:sacrifice) -is:gamechanger -o:add
current index 31, total cards 31, percentage 100.0


In [18]:
import random
print('getting land base')
def pick(cards, n):
    return random.sample(cards, min(n, len(cards)))


chosen_fetches    = pick(fetch_lands, 10)

land_base = []
basic_lands = []
chosen_multicolor = []
if len(color_identity) == 1:
    print('getting only basics')
    basic_lands       = get_basic_lands(color_identity, 28)
elif len(color_identity) == 2:
    print('getting multi lands and basics')
    basic_lands       = get_basic_lands(color_identity, 16)
    chosen_multicolor = pick(multicolor_lands, 12)
elif len(color_identity) == 3:
    print('getting multi lands and basics')
    basic_lands       = get_basic_lands(color_identity, 8)
    chosen_multicolor = pick(multicolor_lands, 20)
elif len(color_identity) >= 4:
    print('getting multi lands and basics')
    basic_lands       = get_basic_lands(color_identity, 4)
    chosen_multicolor = pick(multicolor_lands, 24)

# Basics (names only)
for b in basic_lands:
    land_base.append({"name": b})

# Multicolor lands
land_base.extend(chosen_multicolor)

# Fetch lands
land_base.extend(chosen_fetches)

getting land base
getting multi lands and basics


In [19]:
SOL_RING = "Sol Ring"

In [20]:
def random_filler_query(ci):
    ci_filter = f"ci<={''.join(ci)}" if ci else "ci=0"

    return (
        f"legal:commander "
        f"{ci_filter} "
        f"-type:land "
        f"-is:funny "
        f"-set:UNF "
        f"-set:SUNF "
        f"-o:snow "
        f"-is:playtest "
        f"-type:conspiracy "
        f"-type:emblem "
        f"-name:\"Sol Ring\" "
        f"usd<=1"
    )

print("fetching filler cards")
filler_cards = fetch_all_cards(random_filler_query(color_identity))
random_cards = random.sample(filler_cards, min(30, len(filler_cards)))


fetching filler cards
legal:commander ci<=GRW -type:land -is:funny -set:UNF -set:SUNF -o:snow -type:conspiracy -type:emblem -name:"Sol Ring" usd<=1
current index 175, total cards 14037, percentage 1.246705136425162
current index 350, total cards 14037, percentage 2.493410272850324
current index 525, total cards 14037, percentage 3.7401154092754867
current index 700, total cards 14037, percentage 4.986820545700648
current index 875, total cards 14037, percentage 6.23352568212581
current index 1050, total cards 14037, percentage 7.480230818550973
current index 1225, total cards 14037, percentage 8.726935954976135
current index 1400, total cards 14037, percentage 9.973641091401296
current index 1575, total cards 14037, percentage 11.220346227826457
current index 1750, total cards 14037, percentage 12.46705136425162
current index 1925, total cards 14037, percentage 13.713756500676782
current index 2100, total cards 14037, percentage 14.960461637101947
current index 2275, total cards 14037,

In [21]:
deck = []
deck.append(commander)
# Core spells
deck.extend(chosen_ramp)
deck.extend(chosen_draw)
deck.extend(chosen_removal)

# Lands
deck.extend(land_base)

# Random filler
deck.extend(random_cards)

# Sol Ring
deck.append({"name": SOL_RING})

In [22]:
from collections import Counter

counts = Counter(c["name"] for c in deck)

for name, count in sorted(counts.items()):
    print(f"{count} {name}")

1 Altar of the Lost
1 Angelfire Crusader
1 Arid Mesa
1 Atarka Monument
1 Aura Blast
1 Bomat Bazaar Barge
1 Brass Secretary
1 Brushland
1 Circle of Elders
1 Craterize
1 Deadly Riposte
1 Decommission
1 Diplomatic Relations
1 Discerning Financier
1 Dwarven Miner
1 Elvish Skysweeper
1 Escape Tunnel
1 Eye of Ugin
1 Fever Charm
1 Flood Plain
3 Forest
1 Frog-Squirrels
1 Frontier Guide
1 Gabriel Angelfire
1 Gallifrey Council Chamber
1 Gilded Ghoda
1 Glorious Charge
1 Gongaga, Reactor Town
1 Gore-House Chainwalker
1 Grasslands
1 Gruul Cluestone
1 Guildmages' Forum
1 Hall
1 Hickory Woodlot
1 Horizon of Progress
1 Hormagaunt Horde
1 Hushwood Verge
1 Immersturm Raider
1 Inspired Charge
1 Instrument of the Bards
1 Interdimensional Web Watch
1 Irencrag Feat
1 Iroh, Tea Master
1 Keeper of the Light
1 Klothys, God of Destiny
1 Knight of New Alara
1 Mech Hangar
1 Miner's Bane
1 Mossfire Valley
3 Mountain
1 Omen of the Hunt
1 Outpace Oblivion
1 Pentad Prism
1 Pick Your Poison
1 Pillar of the Paruns
2 Pl